In [50]:
import pandas as pd


In [51]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
file_path = '/content/drive/My Drive/Colab Notebooks/task5dataset.csv'


In [53]:
df = pd.read_csv(file_path)

In [54]:
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [55]:
print(df.columns)

Index(['flags', 'instruction', 'category', 'intent', 'response'], dtype='object')


In [56]:
def clean_text(text):
    text = str(text).strip().lower()
    # Add more cleaning steps if needed (remove greetings, special chars)
    return text

df["cleaned_text"] = df["instruction"].apply(clean_text)


Phase 1 (Zero_shot)

In [ ]:
!pip install langchain-openai

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

# Initialize OpenRouter-backed ChatOpenAI model
llm = ChatOpenAI(
    openai_api_key="Plz enter ur api key",  # Or replace with raw string if testing
    openai_api_base="https://openrouter.ai/api/v1",
    model="openai/gpt-4o",
    temperature=0.0
)

# Create prompt template
template = PromptTemplate.from_template("""
You are a support ticket classifier.
Given the following ticket, return the top 3 most relevant tags.
Ticket: {ticket}
Tags:
""")

# Tagging function
def get_zero_shot_tags(ticket_text):
    prompt = template.format(ticket=ticket_text)
    response = llm.invoke(prompt)
    return response.content


In [58]:
df.head()

,flags,instruction,category,intent,response,cleaned_text
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...,question about cancelling order {{order number}}
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...,i have a question about cancelling oorder {{or...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...,i need help cancelling puchase {{order number}}
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...,i need to cancel purchase {{order number}}
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...,"i cannot afford this order, cancel purchase {{..."


In [59]:
sample_df = df.sample(25, random_state=42)  # Pick 25 random rows
sample_df["predicted_tags"] = sample_df["cleaned_text"].apply(get_zero_shot_tags)


In [60]:
sample_df.head()

,flags,instruction,category,intent,response,cleaned_text,predicted_tags
9329,BLZ,I can't talk with a human agent,CONTACT,contact_human_agent,Thank you for reaching out! I'm sorry to hear ...,i can't talk with a human agent,1. Customer Support\n2. Live Chat\n3. Human As...
4160,BLMZ,I have got to locate hte bills from {{Person N...,INVOICE,check_invoice,I'll get right on it! I understand your need t...,i have got to locate hte bills from {{person n...,1. Billing\n2. Invoice\n3. Account Management
18500,BCELM,"I cannot pay, help me to inform of a problem w...",PAYMENT,payment_issue,I apologize for the inconvenience caused by th...,"i cannot pay, help me to inform of a problem w...","Based on the ticket provided, the top 3 most r..."
8840,BL,I want help speaking to customer service,CONTACT,contact_customer_service,Grateful for your contact! I get the sense tha...,i want help speaking to customer service,1. Customer Service\n2. Support Request\n3. Co...
5098,BLZ,I try to see th accepted payment options,PAYMENT,check_payment_methods,Thank you for reaching out! I completely under...,i try to see th accepted payment options,1. Payment Options\n2. Billing Inquiry\n3. Acc...


Phase 2 (Fine Tuning)

In [61]:
from sklearn.preprocessing import LabelEncoder

sample_df = sample_df.dropna(subset=["instruction", "intent"])
texts = list(sample_df["instruction"])
labels = list(sample_df["intent"])

# Encode labels numerically
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)


In [62]:
from datasets import Dataset
dataset = Dataset.from_dict({"text": texts, "label": labels_encoded})


In [63]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)


In [64]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(labels_encoded)))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [66]:
import wandb
wandb.login(key="8bae383d5337cade93d49019f3e306f8dff0edfb")

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="results",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_steps=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer.train()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Step,Training Loss
5,2.654500
10,2.604800
15,2.485600
20,2.391900
25,2.314400


TrainOutput(global_step=25, training_loss=2.490228843688965, metrics={'train_runtime': 388.2807, 'train_samples_per_second': 0.258, 'train_steps_per_second': 0.064, 'total_flos': 13249574707200.0, 'train_loss': 2.490228843688965, 'epoch': 5.0})

In [67]:
preds = trainer.predict(tokenized_dataset["test"])
predicted_labels = le.inverse_transform(preds.predictions.argmax(axis=1))


In [68]:
# Extract test texts used during evaluation
test_texts = dataset["test"]["text"]

# Find matching indices in sample_df
test_indices = sample_df[sample_df["instruction"].isin(test_texts)].index

# Assign fine-tuned predictions only to those test rows
for i, idx in enumerate(test_indices):
    sample_df.loc[idx, "fine_tuned_pred"] = predicted_labels[i]


Phase 3 (Comparison)

In [69]:
comparison_df = sample_df.loc[test_indices, ["instruction", "intent", "predicted_tags", "fine_tuned_pred"]]
print(comparison_df.head())


                                             instruction  \
4384   I don't know how to take a quick look at invoi...   
3798   i do not know how i can see the early terminat...   
17355  could you help me to unsubscribe to the damn n...   
15938        want assistance receiving a refund of money   
24134      I don't know how to change to the pro account   

                        intent  \
4384             check_invoice   
3798    check_cancellation_fee   
17355  newsletter_subscription   
15938               get_refund   
24134           switch_account   

                                          predicted_tags  \
4384       1. Invoice\n2. Quick View\n3. User Assistance   
3798   1. Early Termination\n2. Penalties\n3. Account...   
17355  1. Unsubscribe\n2. Newsletter\n3. Customer Sup...   
15938         1. Refund\n2. Billing\n3. Customer Support   
24134  1. Account Upgrade\n2. Subscription\n3. Accoun...   

                fine_tuned_pred  
4384             delete_account  
3798 

In [70]:
comparison_df.head()

,instruction,intent,predicted_tags,fine_tuned_pred
4384,I don't know how to take a quick look at invoi...,check_invoice,1. Invoice\n2. Quick View\n3. User Assistance,delete_account
3798,i do not know how i can see the early terminat...,check_cancellation_fee,1. Early Termination\n2. Penalties\n3. Account...,contact_customer_service
17355,could you help me to unsubscribe to the damn n...,newsletter_subscription,1. Unsubscribe\n2. Newsletter\n3. Customer Sup...,contact_customer_service
15938,want assistance receiving a refund of money,get_refund,1. Refund\n2. Billing\n3. Customer Support,delete_account
24134,I don't know how to change to the pro account,switch_account,1. Account Upgrade\n2. Subscription\n3. Accoun...,delete_account


In [71]:
def match_score(pred, true):
    return int(true.lower() in pred.lower())

comparison_df["predicted_tags"] = comparison_df.apply(lambda row: match_score(row["predicted_tags"], row["intent"]), axis=1)
comparison_df["fine_tuned_match"] = comparison_df.apply(lambda row: match_score(row["fine_tuned_pred"], row["intent"]), axis=1)


In [72]:
comparison_df.tail()


,instruction,intent,predicted_tags,fine_tuned_pred,fine_tuned_match
4384,I don't know how to take a quick look at invoi...,check_invoice,0,delete_account,0
3798,i do not know how i can see the early terminat...,check_cancellation_fee,0,contact_customer_service,0
17355,could you help me to unsubscribe to the damn n...,newsletter_subscription,0,contact_customer_service,0
15938,want assistance receiving a refund of money,get_refund,0,delete_account,0
24134,I don't know how to change to the pro account,switch_account,0,delete_account,0


In [73]:
print(preds.predictions[0])  # Check raw logits


[-0.0377365   0.03308371 -0.02395859 -0.04952954 -0.02803271  0.19453977
  0.00376678  0.32747066 -0.17357515 -0.53119284  0.14864379  0.05474795
 -0.03903015 -0.35117882]


In [74]:
print(le.classes_)  # See the full label ↔ index mapping
print(le.inverse_transform([7]))  # Get label name for class 7


['check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'delete_account' 'get_invoice' 'get_refund'
 'newsletter_subscription' 'payment_issue' 'registration_problems'
 'switch_account']
['delete_account']


In [75]:
for i in range(5):
    print(preds.predictions[i].argmax(), le.inverse_transform([preds.predictions[i].argmax()])[0])


7 delete_account
5 contact_customer_service
5 contact_customer_service
7 delete_account
7 delete_account


Phase 4(Few shot prompting)

In [76]:
FEW_SHOT_PROMPT = """
You are a support ticket classifier.

Ticket: "I want to deactivate my account"
Tags: ["delete_account"]

Ticket: "Please send me my latest invoice"
Tags: ["get_invoice"]

Ticket: "What payment methods do you support?"
Tags: ["check_payment_methods"]

Ticket: "I can’t register my account on mobile"
Tags: ["registration_problems"]

Ticket: "{ticket}"
Tags:
"""


In [ ]:


from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

# Initialize OpenRouter-backed ChatOpenAI model
llm = ChatOpenAI(
    openai_api_key="Plz enter ur own api key",  # Or replace with raw string if testing
    openai_api_base="https://openrouter.ai/api/v1",
    model="openai/gpt-4o",
    temperature=0.0
)

# Create prompt template
template = PromptTemplate.from_template("""
You are a support ticket classifier.
Given the following ticket, return the top 3 most relevant tags.
Ticket: {ticket}
Tags:
""")

# Tagging function
def get_few_shot_tags(ticket_text):
    few_shot_prompt = FEW_SHOT_PROMPT.format(ticket=ticket_text)
    response = llm.invoke(few_shot_prompt)
    return response.content
    # Same OpenRouter API call as your zero-shot function



In [78]:
sample_df["few_shot"] = sample_df["cleaned_text"].apply(get_few_shot_tags)


In [79]:
sample_df.head()

,flags,instruction,category,intent,response,cleaned_text,predicted_tags,fine_tuned_pred,few_shot
9329,BLZ,I can't talk with a human agent,CONTACT,contact_human_agent,Thank you for reaching out! I'm sorry to hear ...,i can't talk with a human agent,1. Customer Support\n2. Live Chat\n3. Human As...,NaN,"[""contact_support""]"
4160,BLMZ,I have got to locate hte bills from {{Person N...,INVOICE,check_invoice,I'll get right on it! I understand your need t...,i have got to locate hte bills from {{person n...,1. Billing\n2. Invoice\n3. Account Management,NaN,"[""locate_bills""]"
18500,BCELM,"I cannot pay, help me to inform of a problem w...",PAYMENT,payment_issue,I apologize for the inconvenience caused by th...,"i cannot pay, help me to inform of a problem w...","Based on the ticket provided, the top 3 most r...",NaN,"[""payment_issue""]"
8840,BL,I want help speaking to customer service,CONTACT,contact_customer_service,Grateful for your contact! I get the sense tha...,i want help speaking to customer service,1. Customer Service\n2. Support Request\n3. Co...,NaN,"[""contact_support""]"
5098,BLZ,I try to see th accepted payment options,PAYMENT,check_payment_methods,Thank you for reaching out! I completely under...,i try to see th accepted payment options,1. Payment Options\n2. Billing Inquiry\n3. Acc...,NaN,"[""check_payment_methods""]"


In [80]:
sample_df.head()

,flags,instruction,category,intent,response,cleaned_text,predicted_tags,fine_tuned_pred,few_shot
9329,BLZ,I can't talk with a human agent,CONTACT,contact_human_agent,Thank you for reaching out! I'm sorry to hear ...,i can't talk with a human agent,1. Customer Support\n2. Live Chat\n3. Human As...,NaN,"[""contact_support""]"
4160,BLMZ,I have got to locate hte bills from {{Person N...,INVOICE,check_invoice,I'll get right on it! I understand your need t...,i have got to locate hte bills from {{person n...,1. Billing\n2. Invoice\n3. Account Management,NaN,"[""locate_bills""]"
18500,BCELM,"I cannot pay, help me to inform of a problem w...",PAYMENT,payment_issue,I apologize for the inconvenience caused by th...,"i cannot pay, help me to inform of a problem w...","Based on the ticket provided, the top 3 most r...",NaN,"[""payment_issue""]"
8840,BL,I want help speaking to customer service,CONTACT,contact_customer_service,Grateful for your contact! I get the sense tha...,i want help speaking to customer service,1. Customer Service\n2. Support Request\n3. Co...,NaN,"[""contact_support""]"
5098,BLZ,I try to see th accepted payment options,PAYMENT,check_payment_methods,Thank you for reaching out! I completely under...,i try to see th accepted payment options,1. Payment Options\n2. Billing Inquiry\n3. Acc...,NaN,"[""check_payment_methods""]"


In [81]:
def match_score(pred, true):
    if pd.isna(pred) or pd.isna(true):
        return 0
    return int(true.lower() in str(pred).lower())


In [83]:
# Score Zero-Shot
sample_df["zero_shot_match"] = sample_df.apply(
    lambda row: match_score(row["predicted_tags"], row["intent"]), axis=1
)

# Score Fine-Tuned
sample_df["fine_tuned_match"] = sample_df.apply(
    lambda row: match_score(row["fine_tuned_pred"], row["intent"]), axis=1
)

# Score Few-Shot (assuming you’ve run it and stored predictions)
sample_df["few_shot_match"] = sample_df.apply(
    lambda row: match_score(row["few_shot"], row["intent"]), axis=1
)


In [84]:
print("Zero-Shot Accuracy:", sample_df["zero_shot_match"].mean())
print("Fine-Tuned Accuracy:", sample_df["fine_tuned_match"].mean())
print("Few-Shot Accuracy:", sample_df["few_shot_match"].mean())


Zero-Shot Accuracy: 0.04
Fine-Tuned Accuracy: 0.0
Few-Shot Accuracy: 0.28
